In [1]:
using JuMP, Plasmo,MPI

MPI.Init()
xi = [[7,7] [11,11] [13,13]]

# create empty Plasmo model
graph = GraphModel()
master = Model()
master_node = add_node(graph,master)

@variable(master, 0 <= x[i=1:2] <= 5, Int)
@objective(master, Min, -1.5 * x[1] - 4 * x[2])
children_nodes = Array{NodeOrEdge}(3)
for s = 1:3
    # create a JuMP.Model block linked to m with id s and probability 1/3
    blk = Model()
    child_node = add_node(graph,blk)
    children_nodes[s] = child_node
    @variable(blk, y[j=1:4], Bin)
    @objective(blk, Min, -16 * y[1] + 19 * y[2] + 23 * y[3] + 28 * y[4])
    @linkconstraint(graph, 2 * y[1] + 3 * y[2] + 4 * y[3] + 5 * y[4] <= xi[1,s] - master[:x][1])
    @linkconstraint(graph, 6 * y[1] + y[2] + 3 * y[3] + 2 * y[4] <= xi[2,s] - master[:x][2])
end

solve_types = [:Dual, :Benders, :Extensive]
status = dsp_solve(graph,master_node,children_nodes,solve_type = solve_types[2])  #probabilities are 1/NS by default

# get optimal objective value
getobjectivevalue(master)

# get first-stage variables
println(getvalue(master[:x]))

# get recourse veriables
for node in children_nodes
   println(getvalue(node[:y]))
end

MPI.Finalize()

Finding a good lower bound using Dual Decomposition...

DUAL DECOMPOSITION ITERATION INFORMATION:
* master   = objective function value of the master problem.
* primobj  = best primal objective function value.
* dualobj  = best dual objective function value.
* a.gap(%) = Approximate gap between master and dualobj.
* d.gap(%) = Duality gap between primobj and dualobj.
* times    = wall clock time in seconds.

  iter         master        primobj        dualobj  a.gap(%)  d.gap(%)    time
 D   0  -3.994444e+01          Large  -3.994444e+01      0.00    100.00     0.0
Tr  STOP with gap tolerance +3.055334e-13 (0.00%).
Best lower bound -3.994444e+01, time elapsed: 0.00 sec.
presolving (3 rounds):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 2 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3 variables (0 bin, 2 int, 0 impl, 1 cont) and 2 constraints

 time | node  | left  |LP iter|LP it/n| mem |mdpt |